In [1]:
import pandas as pd
import numpy as np

class BidirectionalHeteroAssociative(object):
    def __init__(self, data_train, data_target, treshold):
        self.train_data = []
        data_target = np.array(data_target)
        self.treshold = treshold
        self.train_data.append([self.make_bipolar(data_train),self.make_bipolar(data_target)])
        self.len_y = len(data_target[0])
        self.weight = np.zeros((data_train.shape[1],data_target.shape[1]),np.int8)
        self.train_association()

    def train_association(self):
        for matriks_pair in self.train_data:
            X = matriks_pair[0]
            Y = matriks_pair[1]
            for data_train, target_train in zip(X,Y):
              self.weight += np.dot(data_train.reshape(len(data_train),1),target_train.reshape(1,len(target_train)))

    def test_association(self, test_data):
        print(test_data)
        test_data = self.calculate_threshold(test_data)
        matriks_result = [0] * self.len_y
        print('Bobot Test:')
        for x in range(1):
            for y in range(self.len_y):
                matriks_result[x] = matriks_result[x] + (test_data[y] * self.weight[y][x])
                print(matriks_result[x],end=" ")
        print()
        print("Hasil Testing")
        return self.calculate_threshold(matriks_result)

    def get_weight_matrix(self):
        return self.weight

    def calculate_threshold(self, vector):
        return [1 if element > self.treshold else element if element == self.treshold else -1 for element in vector]
    
    def make_bipolar(self, vector):
        vector_bipolar = []
        for i in range(len(vector)):
            temp = []
            for j in range(len(vector[i])):
                if vector[i][j] > self.treshold:
                    temp.append(1)
                elif vector[i][j] == self.treshold:
                    temp.append(vector[i][j])
                else:
                    temp.append(-1)
            vector_bipolar.append(temp)
        vector_bipolar = np.array(vector_bipolar)
        return vector_bipolar

if __name__ == '__main__':
    data = pd.read_csv("data_hetero90.csv",sep=';')
    raw_train = np.asarray(data.values[1:26,0:6],float)
    data_target_train = data.values[1:26,6]
    
    target1 = [-1,-1,-1]
    target2 = [-1,1,-1]
    target3 = [1,1,1]
    
    threshold = 0.1

    target_train = [target1 if element == '1' else target2 if element == '2' else target3 for element in data_target_train]
    b = BidirectionalHeteroAssociative(raw_train, target_train, threshold)
    print('\n')
    print('Threshold = ', threshold)
    print('Matrix Bobot: ')
    for i,angka in enumerate(b.get_weight_matrix()):
        for j in range(len(b.get_weight_matrix()[i])):
            print(b.get_weight_matrix()[i][j],end=" ")
        print()
    
    print()
    #testing dengan data testing
    raw_test = np.asarray(data.values[1:7,8:14],float)
    data_target_test = data.values[1:7,14]

    print('data target = ', data_target_test)
    target_test = [target1 if element == '1' else target2 if element == '2' else target3 for element in data_target_test]
    print('target test (bipolar)= ',target_test)
    print()
    print('Test Dengan Data Testing')
    print('----------------------------------------------------------------------------------------------------------')

    count_true = 0
    for i in range(len(raw_test)):
      predicted = b.test_association(raw_test[i])
      real_value = target_test[i]
      validation = np.array_equal(predicted, real_value)
      print(predicted)
      print('Hasil Sebenarnya')
      print(real_value)
      print(validation, '\n')
      if validation == True: 
        count_true+=1

      print('========================================================================================================')
    print ('\nakurasi =', count_true/len(target_test))



Threshold =  0.1
Matrix Bobot: 
-7 5 -7 
9 -7 9 
-9 7 -9 
-9 7 -9 
-7 9 -7 
9 -7 9 

data target =  ['2' '2' '1' '1' '3' '3']
target test (bipolar)=  [[-1, 1, -1], [-1, 1, -1], [-1, -1, -1], [-1, -1, -1], [1, 1, 1], [1, 1, 1]]

Test Dengan Data Testing
----------------------------------------------------------------------------------------------------------
[1.         0.05389023 0.35931052 0.94398995 0.98594359 0.00327491]
Bobot Test:
-7 -16 -25 
Hasil Testing
[-1, -1, -1]
Hasil Sebenarnya
[-1, 1, -1]
False 

[0.79560418 0.06062716 0.38863038 0.94702461 0.88682733 0.00410939]
Bobot Test:
-7 -16 -25 
Hasil Testing
[-1, -1, -1]
Hasil Sebenarnya
[-1, 1, -1]
False 

[0.70326826 0.06778131 0.40330452 0.93537207 0.84842866 0.0050745 ]
Bobot Test:
-7 -16 -25 
Hasil Testing
[-1, -1, -1]
Hasil Sebenarnya
[-1, -1, -1]
True 

[0.7007694  0.06831331 0.39942319 0.93364619 0.0220636  0.00504401]
Bobot Test:
-7 -16 -25 
Hasil Testing
[-1, -1, -1]
Hasil Sebenarnya
[-1, -1, -1]
True 

[0.59149834 0.